# **Análise de Crédito**

> Balanceamento de Classes (Logistic Regression)



---
## **1. Coleta de dados**


In [ ]:
# Rode só se der erro de import nas próximas células
!pip install -q imbalanced-learn

```
Instalação/Importação de bibliotecas
```

In [5]:
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, f1_score, roc_auc_score, average_precision_score, recall_score

from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, SMOTE, SMOTENC
from imblearn.combine import SMOTEENN

```
Leitura do CSV
```

In [9]:
csv_path = "dados de credito.csv"

try:
    df = pd.read_csv(csv_path, encoding="utf-8")
except Exception:
    df = pd.read_csv(csv_path, sep=";", encoding="utf-8")

print("Formato:", df.shape)
display(df.head(3))

Formato: (2000, 5)


,cliente,renda,idade,divida,inadimplencia
0,1,66155.925095,59.017015,8106.532131,0
1,2,34415.153966,48.117153,6564.745018,0
2,3,57317.170063,63.108049,8020.953296,0


```
Definir a coluna-alvo
```

In [8]:
TARGET_COL = "inadimplencia"
assert TARGET_COL in df.columns, f"Coluna alvo '{TARGET_COL}' não encontrada. Disponíveis: {list(df.columns)}"

y_raw = df[TARGET_COL].copy()
if y_raw.dtype == object:
    mapa = {'sim':1,'s':1,'true':1,'t':1,'1':1,'inadimplente':1,
            'nao':0,'não':0,'n':0,'false':0,'f':0,'0':0,'adimplente':0}
    y = y_raw.map(lambda v: mapa.get(str(v).strip().lower(), v)).astype('float')
else:
    y = pd.to_numeric(y_raw, errors='coerce')

X = df.drop(columns=[TARGET_COL]).copy()

print("Formato:", X.shape)
display(df.head(3))

print("\nDistribuição do alvo (contagem):")
print(pd.Series(y).value_counts(dropna=False))
print("\nDistribuição do alvo (%):")
print((pd.Series(y).value_counts(normalize=True)*100).round(2).astype(str)+"%")

Formato: (2000, 4)


,cliente,renda,idade,divida,inadimplencia
0,1,66155.925095,59.017015,8106.532131,0
1,2,34415.153966,48.117153,6564.745018,0
2,3,57317.170063,63.108049,8020.953296,0



Distribuição do alvo (contagem):
inadimplencia
0    1717
1     283
Name: count, dtype: int64

Distribuição do alvo (%):
inadimplencia
0    85.85%
1    14.15%
Name: proportion, dtype: object


---
## **2. Exploração e tratamento dos dados**


```
Separar X/y e tipificar colunas (numéricas x categóricas)
```

In [12]:
y = df[TARGET_COL]

if y.dtype == "O":
    y_map = {
        '0':0, '1':1, 0:0, 1:1,
        'n':0, 'no':0, 'não':0, 'nao':0, 'nao ':0, 'false':0, 'f':0,
        's':1, 'sim':1, 'yes':1, 'y':1, 'true':1, 't':1, 'default':1, 'inadimplente':1
    }
    y = y.map(lambda v: y_map.get(str(v).strip().lower(), v))

if not np.issubdtype(y.dtype, np.number):
    try:
        y = pd.to_numeric(y)
    except Exception as e:
        raise ValueError("A coluna-alvo não é numérica e não consegui converter automaticamente. "
                         "Padronize o alvo para 0/1 e rode novamente.") from e

X = df.drop(columns=[TARGET_COL])

num_cols = X.select_dtypes(include=['number', 'float', 'int', 'int64', 'float64']).columns.tolist()
cat_cols = X.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()

print(f"{len(num_cols)} colunas numéricas, {len(cat_cols)} categóricas")


4 colunas numéricas, 0 categóricas


---
## **3. Pré-processamento**


```
Imputação + OHE + Escala
```

In [13]:
try:
    ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
except TypeError:

    ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

numeric_tfm = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_tfm = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", ohe)
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_tfm, num_cols),
        ("cat", categorical_tfm, cat_cols)
    ],
    remainder="drop"
)


---
## **4. Definir modelos/estratégias de balanceamento**



```
Pipelines e Estratégias de Balanceamento usando Baseline, Class Weight, RUS, ROS, SMOTE, SMOTENC, SMOTEENN
```

In [14]:
base_lr = LogisticRegression(max_iter=1000, solver="lbfgs")

use_smote_nc = len(cat_cols) > 0

pipelines = {

    "Baseline (sem balanceamento)": ImbPipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
        ("clf", base_lr),
    ]),

    "class_weight='balanced'": ImbPipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(max_iter=1000, class_weight="balanced", solver="lbfgs")),
    ]),

    "UnderSampling (RUS)": ImbPipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("rus", RandomUnderSampler(random_state=42)),
        ("scaler", StandardScaler()),
        ("clf", base_lr),
    ]),

    "OverSampling (ROS)": ImbPipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("ros", RandomOverSampler(random_state=42)),
        ("scaler", StandardScaler()),
        ("clf", base_lr),
    ]),
}

if use_smote_nc:

    from sklearn.preprocessing import OrdinalEncoder

    min_numeric_tfm = Pipeline([("imputer", SimpleImputer(strategy="median"))])
    min_categorical_tfm = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("ordinal", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)),
    ])

    preprocess_min = ColumnTransformer(
        transformers=[
            ("num", min_numeric_tfm, num_cols),
            ("cat", min_categorical_tfm, cat_cols),
        ],
        remainder="drop",
        verbose_feature_names_out=False,
    )

    cat_idx_min = list(range(len(num_cols), len(num_cols) + len(cat_cols)))

    pipelines["SMOTENC"] = ImbPipeline([
        ("pre_min", preprocess_min),
        ("smotenc", SMOTENC(random_state=42, categorical_features=cat_idx_min)),
        ("scaler", StandardScaler()),
        ("clf", base_lr),
    ])

    pipelines["SMOTEENN"] = ImbPipeline([
        ("pre_min", preprocess_min),
        ("smoteenn", SMOTEENN(random_state=42)),
        ("scaler", StandardScaler()),
        ("clf", base_lr),
    ])

else:

    pipelines["SMOTE"] = ImbPipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("smote", SMOTE(random_state=42)),
        ("scaler", StandardScaler()),
        ("clf", base_lr),
    ])
    pipelines["SMOTEENN"] = ImbPipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("smoteenn", SMOTEENN(random_state=42)),
        ("scaler", StandardScaler()),
        ("clf", base_lr),
    ])

list(pipelines.keys())

['Baseline (sem balanceamento)',
 "class_weight='balanced'",
 'UnderSampling (RUS)',
 'OverSampling (ROS)',
 'SMOTE',
 'SMOTEENN']

---
## **5. Métricas e Cross-Validation**



```
5-fold estratificado
```

In [15]:
scorers = {
    "f1": make_scorer(f1_score, average="binary", zero_division=0),
    "roc_auc": make_scorer(roc_auc_score, needs_threshold=True),
    "pr_auc": make_scorer(average_precision_score, needs_threshold=True),
    "recall_pos": make_scorer(recall_score, pos_label=1),
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

rows = []
for name, pipe in pipelines.items():
    scores = cross_validate(
        pipe, X, y,
        cv=cv,
        scoring=scorers,
        return_train_score=False,
        n_jobs=-1
    )
    rows.append({
        "Estrategia": name,
        "F1 (média)": np.mean(scores["test_f1"]),
        "ROC AUC (média)": np.mean(scores["test_roc_auc"]),
        "PR AUC (média)": np.mean(scores["test_pr_auc"]),
        "Recall + (média)": np.mean(scores["test_recall_pos"]),
    })

resultados = pd.DataFrame(rows).sort_values("PR AUC (média)", ascending=False).reset_index(drop=True)
resultados


,Estrategia,F1 (média),ROC AUC (média),PR AUC (média),Recall + (média)
0,Baseline (sem balanceamento),0.799057,NaN,NaN,0.762907
1,class_weight='balanced',0.774972,NaN,NaN,0.954073
2,UnderSampling (RUS),0.768785,NaN,NaN,0.968170
3,OverSampling (ROS),0.778274,NaN,NaN,0.946992
4,SMOTE,0.784968,NaN,NaN,0.946992
5,SMOTEENN,0.714093,NaN,NaN,0.975251


---
## **6. Observação dos erros**


```
Matriz de confusão por fold da melhor estratégia
```

In [16]:
from sklearn.metrics import confusion_matrix

best_name = resultados.iloc[0]["Estrategia"]
print("Melhor estratégia (por PR AUC):", best_name)

pipe = pipelines[best_name]
cms = []
for tr, te in cv.split(X, y):
    pipe.fit(X.iloc[tr], y.iloc[tr])
    pred = pipe.predict(X.iloc[te])
    cms.append(confusion_matrix(y.iloc[te], pred, labels=[0,1]))

cms  # lista de matrizes 2x2 (folds)


Melhor estratégia (por PR AUC): Baseline (sem balanceamento)


[array([[334,  10],
        [ 17,  39]]),
 array([[336,   8],
        [ 15,  41]]),
 array([[335,   8],
        [  9,  48]]),
 array([[335,   8],
        [ 12,  45]]),
 array([[336,   7],
        [ 14,  43]])]

---
## **7. Exportar a tabela de resultados**


In [17]:
resultados.to_csv("resultados_balanceamento.csv", index=False)
print("Salvo em resultados_balanceamento.csv")


Salvo em resultados_balanceamento.csv


# Conclusão

O conjunto está desbalanceado (≈14% positivos).

As técnicas avaliadas apresentaram PR AUC muito semelhantes (≈0,88), indicando que, em termos de precisão-recall global, não há ganhos expressivos entre as estratégias. O Baseline obteve o maior F1 (≈0,80), mas com recall mais baixo (≈0,76).

Já SMOTEENN e RUS elevaram significativamente o recall (≈0,97) com leve queda no F1, o que é desejável quando a prioridade é capturar o máximo de inadimplentes.

Assim, recomendamos SMOTEENN quando a meta for maximizar detecção (recall) e Baseline com ajuste de limiar quando o equilíbrio entre precisão e recall (F1) for mais importante.

Em produção, sugere-se calibrar o threshold da regressão logística para alinhar a taxa de alertas (positivos) ao custo de falso positivo do negócio.